<b>Import of required libraries</b>

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import shapiro, bartlett, f_oneway, kruskal, friedmanchisquare
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison, pairwise_tukeyhsd
from statsmodels.formula.api import ols

<b>1. One-way ANOVA</b>

In [4]:
def check_sample_normality(sample, alpha=0.05):
    # check if sample comes from gaussian distribution (Shapiro-Wilk test)
    stat, p = shapiro(sorted(sample))

    alpha = alpha

    print('\n=== Shapiro-Wilk normality test ===',
          '\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

    if p <= alpha:
        print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
    else:
        print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis')    

def check_samples_equal_variance(sample_1, sample_2, sample_3, alpha=0.05):
    # check if the variance in every sample is simmilar enough (Bartlett test)
    stat, p = bartlett(sample_1, sample_2, sample_3)
    
    print('\n=== Bartlett test for the simmilarity of variances samples ===',
          '\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

    if p <= alpha:
        print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
    else:
        print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis') 

# checking if ANOVA can be conducted
def check_anova_assumptions(sample_1, sample_2, sample_3, alpha=0.05):
    
    # check if every sample comes from gaussian distribution
    check_sample_normality(sample_1)
    check_sample_normality(sample_2)
    check_sample_normality(sample_3)
    
    # check if the variance in each sample is simmilar enough (Bartlett test)
    check_samples_equal_variance(sample_1, sample_2, sample_3)

mat_file = loadmat(os.path.join('anova_data', 'anova_data.mat'))

df = pd.DataFrame(mat_file['koala'])

check_anova_assumptions(df[0], df[1], df[2])

stat, p = f_oneway(df[0], df[1], df[2])
alpha = 0.05
    
print('\n=== One-way ANOVA test for 3 samples ===', 
      '\nH0: \t\t\t Every sample comes from distribution with simmilar mean', '\nH1: \t\t\t At least one sample has different mean than the others \n\t\t\t (what results in a different mean of its\' population)', 
      '\n\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

if p <= alpha:
    print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
else:
    print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis') 



=== Shapiro-Wilk normality test === 
p-value: 		 0.9137757420539856
alpha: 			 0.05

Result: 		 p-value is greater than alpha 
 			 We can't reject null hypothesis

=== Shapiro-Wilk normality test === 
p-value: 		 0.5149158239364624
alpha: 			 0.05

Result: 		 p-value is greater than alpha 
 			 We can't reject null hypothesis

=== Shapiro-Wilk normality test === 
p-value: 		 0.975721001625061
alpha: 			 0.05

Result: 		 p-value is greater than alpha 
 			 We can't reject null hypothesis

=== Bartlett test for the simmilarity of variances samples === 
p-value: 		 0.7816443631023369
alpha: 			 0.05

Result: 		 p-value is greater than alpha 
 			 We can't reject null hypothesis

=== One-way ANOVA test for 3 samples === 
H0: 			 Every sample comes from distribution with simmilar mean 
H1: 			 At least one sample has different mean than the others 
			 (what results in a different mean of its' population) 

p-value: 		 0.06903397098047648
alpha: 			 0.05

Result: 		 p-value is greater tha

<b>2. One-way ANOVA and then Tukey's HSD test to check pairwise which sample doesn't have mean equal to others</b>

In [23]:
mat_file = loadmat(os.path.join('anova_data', 'anova_data.mat'))

df = pd.DataFrame()

df['hrs_of_activity'] = pd.Series(np.reshape(mat_file['wombats'], 
                                             (mat_file['wombats'].shape[1])))
df['group'] = pd.Series(np.reshape(mat_file['wombat_groups'], 
                                   (mat_file['wombat_groups'].shape[1])))

stat, p = f_oneway(df[df['group'] == 1]['hrs_of_activity'],
                   df[df['group'] == 2]['hrs_of_activity'],
                   df[df['group'] == 3]['hrs_of_activity'])
alpha = 0.05
    
print('\n=== One-way ANOVA test for 3 samples ===', 
      '\nH0: \t\t\t Every sample comes from distribution with simmilar mean', 
      '\nH1: \t\t\t At least one sample has different mean than the others \n\t\t\t (what results in a different mean of its\' population)', 
      '\n\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

if p <= alpha:
    print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
else:
    print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis') 
    
# Tukey's HSD test
pairwise_tukeyhsd(df['hrs_of_activity'], df['group'], alpha=0.05).summary()


=== One-way ANOVA test for 3 samples === 
H0: 			 Every sample comes from distribution with simmilar mean 
H1: 			 At least one sample has different mean than the others 
			 (what results in a different mean of its' population) 

p-value: 		 0.04338747516521317
alpha: 			 0.05

Result: 		 p-value is smaller than or equal to alpha 
 			 We reject null hypothesis


group1,group2,meandiff,p-adj,lower,upper,reject
1,2,4.2893,0.0339,0.2892,8.2893,True
1,3,1.8519,0.4528,-1.9194,5.6232,False
2,3,-2.4374,0.3004,-6.4374,1.5627,False


<b>3. Kruskal-Wallis and Friedman tests for samples for which ANOVA can't be used</b>

In [6]:
kw_1 = [3415, 1593, 1976, 1526, 1538, 983, 1050, 1861, 1714, 1320, 1276, 1263, 1271, 1436]
kw_2 = [4556, 1937, 2056, 1594, 1634, 1086, 1209, 2087, 2415, 1621, 1377, 1279, 1417, 1310]
kw_3 = [5772, 2242, 2240, 1644, 1866, 1135, 1245, 2054, 2361, 1624, 1522, 1350, 1583, 1357]
kw_4 = [5432, 2794, 2085, 1705, 1769, 1177, 977, 2018, 2424, 1551, 1412, 1490, 1513, 1468]

df = pd.DataFrame({'Kwartał I': kw_1,'Kwartał II': kw_2,'Kwartał III': kw_3,'Kwartał IV': kw_4})

stat, p = kruskal(df['Kwartał I'], df['Kwartał II'], df['Kwartał III'], df['Kwartał IV'])

alpha = 0.05
    
print('\n=== Kruskal-Wallis test for 4 samples ===', 
      '\n\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

if p <= alpha:
    print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
else:
    print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis') 

stat, p = friedmanchisquare(df['Kwartał I'], df['Kwartał II'], df['Kwartał III'], df['Kwartał IV'])

alpha = 0.05
    
print('\n=== Friedman test for 4 samples ===', 
      '\n\np-value: \t\t {0}\nalpha: \t\t\t {1}\n'.format(p, alpha))

if p <= alpha:
    print('Result: \t\t p-value is smaller than or equal to alpha \n \t\t\t We reject null hypothesis')
else:
    print('Result: \t\t p-value is greater than alpha \n \t\t\t We can\'t reject null hypothesis') 



=== Kruskal-Wallis test for 4 samples === 

p-value: 		 0.44062901941658017
alpha: 			 0.05

Result: 		 p-value is greater than alpha 
 			 We can't reject null hypothesis

=== Friedman test for 4 samples === 

p-value: 		 2.6030689788696133e-05
alpha: 			 0.05

Result: 		 p-value is smaller than or equal to alpha 
 			 We reject null hypothesis


<b>4. Tukey's HSD test to check pairwise which sample from example above doesn't have mean equal to others</b>

In [20]:
kw_1 = [3415, 1593, 1976, 1526, 1538, 983, 1050, 1861, 1714, 1320, 1276, 1263, 1271, 1436]
kw_2 = [4556, 1937, 2056, 1594, 1634, 1086, 1209, 2087, 2415, 1621, 1377, 1279, 1417, 1310]
kw_3 = [5772, 2242, 2240, 1644, 1866, 1135, 1245, 2054, 2361, 1624, 1522, 1350, 1583, 1357]
kw_4 = [5432, 2794, 2085, 1705, 1769, 1177, 977, 2018, 2424, 1551, 1412, 1490, 1513, 1468]


df = pd.DataFrame({'Kwartał I': kw_1,'Kwartał II': kw_2,'Kwartał III': kw_3,'Kwartał IV': kw_4})

stacked_data = df.stack().reset_index()
stacked_data = stacked_data.rename(columns={'level_0': 'id',
                                            'level_1': 'Kwartał',
                                            0:'Wynik'})

pairwise_tukeyhsd(stacked_data['Wynik'], stacked_data['Kwartał'], alpha=0.05).summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Kwartał I,Kwartał II,239.7143,0.9,-720.8531,1200.2817,False
Kwartał I,Kwartał III,412.3571,0.6489,-548.2103,1372.9246,False
Kwartał I,Kwartał IV,399.5,0.6686,-561.0674,1360.0674,False
Kwartał II,Kwartał III,172.6429,0.9,-787.9246,1133.2103,False
Kwartał II,Kwartał IV,159.7857,0.9,-800.7817,1120.3531,False
Kwartał III,Kwartał IV,-12.8571,0.9,-973.4246,947.7103,False


<b>5. Two-way ANOVA</b>